# Transfer Learning - II

Using trained model weights to improve training on segmented augmented data

## Imports

In [ ]:
# import keras
# Source for HAM10000: https://www.kaggle.com/alexako/cs200-1-u-net-skin-lesion-segmentation

import cv2
import numpy as np
import pandas as pd
import os
import random

In [ ]:
from keras.models import Model, Sequential, load_model
from keras.layers import Activation, Dense, BatchNormalization, Dropout, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, Input, Reshape, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras.optimizers import adam_v2 #, SGD
import tensorflow as tf

In [ ]:
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from warnings import filterwarnings

## Colab Utils

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
root_folder = r"/content/drive/MyDrive/Spring 2022/Machine Learning for Physical Applications/Project/Manas Trials/Self-Supervised/"
dataset_folder = r"/content/drive/MyDrive/Spring 2022/Machine Learning for Physical Applications/Project/dataset_anemia/Final_Augmented_Data/"

## Load Data

In [ ]:
italy_data = pd.read_excel(r"/content/drive/MyDrive/Spring 2022/Machine Learning for Physical Applications/Project/TRIAL/Italy_new.xlsx")
india_data = pd.read_excel(r"/content/drive/MyDrive/Spring 2022/Machine Learning for Physical Applications/Project/dataset_anemia/India/India.xlsx")

italy_data['Source'] = "Italy"
india_data["Source"] = "India" 

# italy_skip = [92]
# italy_data.drop(italy_skip,0,inplace=True)

india_root = r"/content/drive/MyDrive/Spring 2022/Machine Learning for Physical Applications/Project/dataset_anemia/India/"
italy_root = r"/content/drive/MyDrive/Spring 2022/Machine Learning for Physical Applications/Project/dataset_anemia/Italy/"

# india_data['Path'] = india_root + india_data['Number'].astype('str')

df = pd.concat([italy_data, india_data])

df['Number'] = df['Number'].astype('int').astype('str')
df['Hgb'] = df['Hgb'].astype('float')
df['Age'] = df['Age'].astype('float')

df.loc[df[df.Gender == 'F'].index, 'Gender'] = 1.0
df.loc[df[df.Gender == 'M'].index, 'Gender'] = 0.0

df = df[df.Note.isna()][['Number', 'Source', 'Hgb', 'Gender', 'Age']]
df = df.reset_index()

X = []
k = 100
idx = random.choices(df.index, k=k)
# hb = df.Hgb[idx].values
# age = df.Age[idx].values
# gen = df.Gender[idx].values

for i in idx: #list(df.index):
    if df.loc[i, 'Source'] == "Italy":
        file_path = os.path.join(italy_root, df.loc[i, 'Number'])
    else:
        file_path = os.path.join(india_root, df.loc[i, 'Number'])
    files = sorted(os.listdir(file_path))
    img = cv2.imread(os.path.join(file_path, files[0]), 1)
    img = cv2.resize(img, (480, 640))
    X.append(img)

X = np.array(X)/255.0

In [ ]:
y = df.loc[idx,['Hgb']].values/20 # , 'Gender', 'Age']

## Data Augmentation

In [ ]:
def random_rotation(x_image, y_image):
    rows_x,cols_x, chl_x = x_image.shape
    rows_y,cols_y = y_image.shape
    rand_num = np.random.randint(-40,40)
    M1 = cv2.getRotationMatrix2D((cols_x/2,rows_x/2),rand_num,1)
    M2 = cv2.getRotationMatrix2D((cols_y/2,rows_y/2),rand_num,1)
    x_image = cv2.warpAffine(x_image,M1,(cols_x,rows_x))
    y_image = cv2.warpAffine(y_image.astype('float32'),M2,(cols_y,rows_y))
    return x_image, y_image.astype('int')

def horizontal_flip(x_image, y_image):
    x_image = cv2.flip(x_image, 1)
    y_image = cv2.flip(y_image.astype('float32'), 1)
    return x_image, y_image.astype('int')

In [ ]:
def img_augmentation(x_train, y_train):
    x_rotat = []
    y_rotat = []
    x_flip = []
    y_flip = []
    x_nois = []
    for idx in range(len(x_train)):
        x,y = random_rotation(x_train[idx], y_train[idx])
        x_rotat.append(x)
        y_rotat.append(y)
        
        x,y = horizontal_flip(x_train[idx], y_train[idx])
        x_flip.append(x)
        y_flip.append(y)
        return np.array(x_rotat), np.array(y_rotat), np.array(x_flip), np.array(y_flip)

In [ ]:
x_rotated, y_rotated, x_flipped, y_flipped = img_augmentation(x_train, y_train)
x_train_full = np.concatenate([x_train, x_rotated, x_flipped])

In [ ]:
# Clear useless variables
del x_train
del x_rotated
del x_flipped
del y_train
del y_rotated
del y_flipped

## Augment Data

In [ ]:
x_rotated, y_rotated, x_flipped, y_flipped = img_augmentation(X, y)

X_clf = np.concatenate([X, x_rotated, x_flipped])
y_clf = np.concatenate([y, y_rotated, y_flipped])

In [ ]:
X_clf.shape, y_clf.shape

## Model Setup

In [ ]:
def encoder(img_input):
  x = Conv2D(64, (3, 3), padding='same', name='conv1',strides= (1,1))(img_input)
  x = BatchNormalization(name='bn1')(x)
  x = Activation('relu')(x)
  x = Conv2D(64, (3, 3), padding='same', name='conv2')(x)
  x = BatchNormalization(name='bn2')(x)
  x = Activation('relu')(x)
  x = MaxPooling2D()(x)

  x = Conv2D(128, (3, 3), padding='same', name='conv3')(x)
  x = BatchNormalization(name='bn3')(x)
  x = Activation('relu')(x)
  x = Conv2D(128, (3, 3), padding='same', name='conv4')(x)
  x = BatchNormalization(name='bn4')(x)
  x = Activation('relu')(x)
  x = MaxPooling2D()(x)

  x = Conv2D(256, (3, 3), padding='same', name='conv5')(x)
  x = BatchNormalization(name='bn5')(x)
  x = Activation('relu')(x)
  x = Conv2D(256, (3, 3), padding='same', name='conv6')(x)
  x = BatchNormalization(name='bn6')(x)
  x = Activation('relu')(x)
  x = Conv2D(256, (3, 3), padding='same', name='conv7')(x)
  x = BatchNormalization(name='bn7')(x)
  x = Activation('relu')(x)
  x = MaxPooling2D()(x)

  x = Conv2D(512, (3, 3), padding='same', name='conv8')(x)
  x = BatchNormalization(name='bn8')(x)
  x = Activation('relu')(x)
  x = Conv2D(512, (3, 3), padding='same', name='conv9')(x)
  x = BatchNormalization(name='bn9')(x)
  x = Activation('relu')(x)
  x = Conv2D(512, (3, 3), padding='same', name='conv10')(x)
  x = BatchNormalization(name='bn10')(x)
  x = Activation('relu')(x)
  x = MaxPooling2D()(x)

  x = Conv2D(512, (3, 3), padding='same', name='conv11')(x)
  x = BatchNormalization(name='bn11')(x)
  x = Activation('relu')(x)
  x = Conv2D(512, (3, 3), padding='same', name='conv12')(x)
  x = BatchNormalization(name='bn12')(x)
  x = Activation('relu')(x)
  x = Conv2D(512, (3, 3), padding='same', name='conv13')(x)
  x = BatchNormalization(name='bn13')(x)
  x = Activation('relu')(x)
  x = MaxPooling2D()(x)

  x = Dense(1024, activation = 'relu', name='fc1')(x)
  x = Dense(1024, activation = 'relu', name='fc2')(x)

  return x

In [ ]:
def decoder(enc):
  # Decoding Layer 
  x = UpSampling2D()(enc)
  x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv1')(x)
  x = BatchNormalization(name='bn14')(x)
  x = Activation('relu')(x)
  x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv2')(x)
  x = BatchNormalization(name='bn15')(x)
  x = Activation('relu')(x)
  x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv3')(x)
  x = BatchNormalization(name='bn16')(x)
  x = Activation('relu')(x)

  x = UpSampling2D()(x)
  x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv4')(x)
  x = BatchNormalization(name='bn17')(x)
  x = Activation('relu')(x)
  x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv5')(x)
  x = BatchNormalization(name='bn18')(x)
  x = Activation('relu')(x)
  x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv6')(x)
  x = BatchNormalization(name='bn19')(x)
  x = Activation('relu')(x)

  x = UpSampling2D()(x)
  x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv7')(x)
  x = BatchNormalization(name='bn20')(x)
  x = Activation('relu')(x)
  x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv8')(x)
  x = BatchNormalization(name='bn21')(x)
  x = Activation('relu')(x)
  x = Conv2DTranspose(128, (3, 3), padding='same', name='deconv9')(x)
  x = BatchNormalization(name='bn22')(x)
  x = Activation('relu')(x)

  x = UpSampling2D()(x)
  x = Conv2DTranspose(128, (3, 3), padding='same', name='deconv10')(x)
  x = BatchNormalization(name='bn23')(x)
  x = Activation('relu')(x)
  x = Conv2DTranspose(64, (3, 3), padding='same', name='deconv11')(x)
  x = BatchNormalization(name='bn24')(x)
  x = Activation('relu')(x)

  x = UpSampling2D()(x)
  x = Conv2DTranspose(64, (3, 3), padding='same', name='deconv12')(x)
  x = BatchNormalization(name='bn25')(x)
  x = Activation('relu')(x)
  x = Conv2DTranspose(1, (3, 3), padding='same', name='deconv13')(x)
  x = BatchNormalization(name='bn26')(x)
  x = Activation('sigmoid')(x)
  return x


In [ ]:
def unet():
    img_input = Input(shape= (None, None, 3))
    enc = encoder(img_input)
    dec = decoder(enc)
    pred = Reshape((192,256))(dec)
    
    model = Model(inputs=img_input, outputs=pred)
    
    model.compile(optimizer= adam_v2.Adam(learning_rate=0.001), loss= ["binary_crossentropy"])
    # , momentum=0.9, decay=0.0005, nesterov=False)
    print("Model compiled successfully") #model.summary()
    return model

## Load Weights to Model

In [ ]:
base_epochs = 100
weight_path = os.path.join(root_folder,'unet_'+str(base_epochs)+'_epoch.h5')

In [ ]:
model = unet()

In [ ]:
enc_in_tensor = model.input
enc_out_tensor = model.get_layer('bn10').output # fc2

In [ ]:
encoder_model = Model(inputs = enc_in_tensor, outputs = enc_out_tensor)

In [ ]:
encoder_model.load_weights(weight_path, by_name=True)

## Custom Metric

In [ ]:
def rms_loss20(y_true, y_pred):
    y_true_f = 20*K.flatten(y_true)
    y_pred_f = 20*K.flatten(y_pred)
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 

## Generate Secondary Model

In [ ]:
from keras.layers.pooling import GlobalAveragePooling2D
def raw_pred(img_input):
    enc = encoder_model(img_input)
    x = GlobalAveragePooling2D(name='gap0')(enc)
    x = Dense(64, activation='relu', name='pred1')(x)
    pred = Dense(1, activation='sigmoid', name='pred_final')(x)
    model = Model(inputs=img_input, outputs=pred)
    model.compile(loss=['mse'], optimizer=adam_v2.Adam(learning_rate=0.001), metrics=[rms_loss20])
    print("Classifier compiled successfully")
    return model

In [ ]:
def main_model(img_input, age, gen):
  pred = raw_pred(img_input)
  model = Model(inputs=[pred, age, gen])

## Training

In [ ]:
X_clf_use, X_clf_test, y_clf_use, y_clf_test = train_test_split(X_clf, y_clf, test_size = 0.1, random_state = 101)
X_clf_train, X_clf_val, y_clf_train, y_clf_val = train_test_split(X_clf_use, y_clf_use, test_size = 0.1, random_state = 101)

In [ ]:
X_clf_train.shape, X_clf_val.shape, X_clf_test.shape, y_clf_train.shape, y_clf_val.shape, y_clf_test.shape

In [ ]:
def train_clf(epochs_num, savename):
    model =  raw_pred(Input(shape= (None, None, 3))) #epochs_num, savename)
    hist = model.fit(X_clf_use, y_clf_use, epochs= epochs_num, batch_size= 6, validation_data= (X_clf_val, y_clf_val), 
                     verbose=1)
    model.save(savename)
    return model,hist

In [ ]:
epochs = 5
clf_weight_path = os.path.join(root_folder,'clf_'+str(epochs)+'_epoch.h5')
clf_model, hist = train_clf(epochs, clf_weight_path)

In [ ]:
 X_clf_train.shape, y_clf_train.shape

In [ ]:
y_clf_test[2:5]

In [ ]:
rms_loss20(y_clf_test[12:15],clf_model(X_clf_test[12:15]))

In [ ]:
clf_model(X_clf_test[12:15]), y_clf_test[12:15]